# Twitter sentiment analysis using RoBERTa model

In [1]:
!pip install transformers


In [2]:
!pip install torch


In [15]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax

tweet = "@eshwarnadh today's cold @ home 😒 https://eshwar.com"
# tweet = 'Great content! subscribed 😉'

# precprcess tweet
tweet_words = []

for word in tweet.split(' '):
    if word.startswith('@') and len(word) > 1:
        word = '@user'
    
    elif word.startswith('http'):
        word = "http"
    tweet_words.append(word)

tweet_proc = " ".join(tweet_words)

## load model and tokenizer
roberta = "cardiffnlp/twitter-roberta-base-sentiment"
model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)
labels = ['Negative', 'Neutral', 'Positive'] 

# # sentiment analysis
encoded_tweet = tokenizer(tweet_proc, return_tensors='pt')
print(encoded_tweet)
output = model(encoded_tweet['input_ids'], encoded_tweet['attention_mask'])
print(output)
# output = model(**encoded_tweet) another way to write above line

scores = output[0][0].detach().numpy()
scores = softmax(scores)

for i in range(len(scores)):
    
    l = labels[i]
    s = scores[i]
    print(l,s)




{'input_ids': tensor([[    0,  1039, 12105,   452,    18,  2569,   787,   184, 17841, 10659,
          2054,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
SequenceClassifierOutput(loss=None, logits=tensor([[ 1.3710,  0.3350, -1.7215]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
Negative 0.7141528
Neutral 0.25343058
Positive 0.0324166


In [17]:
!pip install gradio

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [19]:
import gradio as gr

In [21]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
def classify_tweet(tweet):
    tweet_words = []

    for word in tweet.split(' '):
        if word.startswith('@') and len(word) > 1:
            word = '@user'

        elif word.startswith('http'):
            word = "http"
        tweet_words.append(word)

    tweet_proc = " ".join(tweet_words)
    roberta = "cardiffnlp/twitter-roberta-base-sentiment"
    model = AutoModelForSequenceClassification.from_pretrained(roberta)
    tokenizer = AutoTokenizer.from_pretrained(roberta)
    labels = ['Negative', 'Neutral', 'Positive'] 

    # # sentiment analysis
    encoded_tweet = tokenizer(tweet_proc, return_tensors='pt')
    output = model(encoded_tweet['input_ids'], encoded_tweet['attention_mask'])
    # output = model(**encoded_tweet) another way to write above line

    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    predictions = {labels[i]:float(scores[i])  for i in range(len(scores))}
#     for i in range(len(scores)):
#         l = labels[i]
#         s = scores[i]
#         predictions[l] = s
    return predictions

In [23]:
interface=gr.Interface(
            fn=classify_tweet,
            inputs="textbox",
            outputs=gr.Label(num_top_classes=3))
interface.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
